In [1]:
from transformers import BertTokenizer,BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.eval()


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [2]:
from PyDictionary import PyDictionary

dictionary=PyDictionary()

word = "departure"


In [4]:
with open('saved_objects/words/top_words.txt') as tf:
    top_common_words = [line.strip() for line in tf.readlines()]
print(len(top_common_words))

199


In [5]:
print (dictionary.meaning(word))

{'Noun': ['the act of departing', 'a variation that deviates from the standard or norm', 'euphemistic expressions for death']}


In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
d = dictionary.meaning(word)
#summary = word
summary = ""
num = 2
for pos in d:
    for a in d[pos][:min(num, len(d[pos]))]:  # update for more pos's? maybe try with 2 not 3 definitions
        num = max(1,num-1)
        words = a.split(" ")
        i = 0
        while len(words[i]) <= 3 or words[i] in [word] or words[i] in summary: i += 1
        summary += " " + words[i]
        j = 1
        while j < len(words) and len(words[-j]) <= 3 or words[-j] in [word] or words[-j] in summary: j += 1
        if j != len(words): summary += " " + words[-j]
print(summary)

 departing variation norm


In [7]:
meaning = ""
for pos in d:
    for sent in d[pos]:
        meaning += stemmer.stem(sent) + " "
print(meaning)

the act of depart a variation that deviates from the standard or norm euphemistic expressions for death 


In [8]:
import torch

tokens = tokenizer.tokenize(summary)
print(tokens)
input_ids = torch.tensor(tokenizer.encode(tokens)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
predictions = outputs[0][0]
print(predictions.shape)

['departing', 'variation', 'norm']
torch.Size([5, 30522])


In [9]:
import nltk
def is_good_word(stri):
    print(stri)
    punctuation = ['-', ':', '+', ',', '～', '`', '!', '.', ',', '?', '*', "\"", "|", "।", "॥"]
    if stri in punctuation or '#' in stri:
        return False
    if stri == word or stri in summary:
        return False
    if stemmer.stem(stri) in meaning:
        return False
    if stri in top_common_words:
        return False
    pos = nltk.pos_tag([stri])[0][1]
    if 'JJ' in pos or 'JJ' in pos: # or 'RB' in pos or 'VB' in pos: #if its a noun, adjective(? or adverb we good just noun for now
        return True
    return False

In [10]:
#with torch.no_grad():
#average = torch.mean(predictions[1:-1], 0) ##improve this!!!
average = torch.median(predictions[1:-1], 0).values
#average = torch.max(predictions[1:-1], 0).values
print(average.shape)
x = sorted(average)
k = 1
best = torch.argmax(average)
while not is_good_word(tokenizer.convert_ids_to_tokens(best.item())):
    k+=1
    y = x[-k]
    best = (average == y).nonzero().flatten()
print(best)
print("The metaphor is", tokenizer.convert_ids_to_tokens(best.item()))

torch.Size([30522])
the
.
,
norm
a
-
in
##s
)
residual
tensor([21961])
The metaphor is residual


In [11]:
#tokenizer.convert_ids_to_tokens(nltk.pos_tag([1024])[0][1]) == ':'
#nltk.pos_tag([tokenizer.convert_ids_to_tokens(1024)])[0][1] == ':'
print(meaning)
stemmer.stem('depature')

the act of depart a variation that deviates from the standard or norm euphemistic expressions for death 


'depatur'

In [12]:
import numpy as np
print(list(np.arange(40)))
tokenizer.convert_ids_to_tokens(np.arange(16000,18000))
#100-103 tags
#the (first word) = 1996

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


['kappa',
 '##hel',
 '80s',
 'strengthening',
 'appealing',
 'brewing',
 'gypsy',
 'mali',
 'lashes',
 'hulk',
 'unpleasant',
 'harassment',
 'bio',
 'treaties',
 'predict',
 'instrumentation',
 'pulp',
 'troupe',
 'boiling',
 'mantle',
 '##ffe',
 'ins',
 '##vn',
 'dividing',
 'handles',
 'verbs',
 '##onal',
 'coconut',
 'senegal',
 '340',
 'thorough',
 'gum',
 'momentarily',
 '##sto',
 'cocaine',
 'panicked',
 'destined',
 '##turing',
 'teatro',
 'denying',
 'weary',
 'captained',
 'mans',
 '##hawks',
 '##code',
 'wakefield',
 'bollywood',
 'thankfully',
 '##16',
 'cyril',
 '##wu',
 'amendments',
 '##bahn',
 'consultation',
 'stud',
 'reflections',
 'kindness',
 '1787',
 'internally',
 '##ovo',
 'tex',
 'mosaic',
 'distribute',
 'paddy',
 'seeming',
 '143',
 '##hic',
 'piers',
 '##15',
 '##mura',
 '##verse',
 'popularly',
 'winger',
 'kang',
 'sentinel',
 'mccoy',
 '##anza',
 'covenant',
 '##bag',
 'verge',
 'fireworks',
 'suppress',
 'thrilled',
 'dominate',
 '##jar',
 'swansea',
 '#

In [13]:
'#' in '##a'
nltk.pos_tag(["quick"])

[('quick', 'NN')]

In [14]:
average = torch.mean(predictions[1:-1], axis = 0)
print(average.shape) #####IMPROVE THIS
from heapq import nlargest
indexes = torch.arange(0,len(average))
n = nlargest(10, indexes, key=lambda i: average[i])
print(n)
tokenizer.convert_ids_to_tokens(n)

torch.Size([30522])
[tensor(1012), tensor(1064), tensor(1025), tensor(8386), tensor(1010), tensor(1996), tensor(1011), tensor(1029), tensor(1998), tensor(1999)]


['.', '|', ';', 'variation', ',', 'the', '-', '?', 'and', 'in']